In [5]:
import gspread
import pandas as pd
import requests
from oauth2client.service_account import ServiceAccountCredentials
from bs4 import BeautifulSoup
from unidecode import unidecode

In [32]:

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name('C:/Users/Jedrek/Downloads/serene-cathode-397013-f6b9fc969ce8.json', scope)
client = gspread.authorize(credentials)

fbref = client.open_by_key('1iqsF5kwcy7hxWpuJXyMniZOrB6iw3hSz56QkgIfWazQ')
transfermakrt = client.open_by_url('https://docs.google.com/spreadsheets/d/1gPFuQpn6CNvK6yysY2w7UDdc9pj2EOALF0Jhd83-xpg/edit#gid=1385039206')


sheet_list = fbref.worksheets()

dataframes = {}
transfermarkt_values = {}


for sheet in sheet_list:
    if sheet.title != 'Plan' and sheet.title != 'Glossary' and sheet.title != 'Mapping' and sheet.title != 'Arkusz30' :
        data = sheet.get_all_values()
        df = pd.DataFrame(data[1:], columns=data[0]) 
        dataframes[sheet.title] = df

sheet_list1 = transfermakrt.worksheets()
data = sheet_list1[0].get_all_values()
transfermarkt_values = pd.DataFrame(data[1:], columns=data[0] )


In [33]:
names = []

transfermarkt_values = transfermarkt_values.query('age != "-"')
transfermarkt_values = transfermarkt_values.reset_index()
for player in transfermarkt_values['player']:
    name = unidecode(player)
    name = name.lower()
    name = name.replace(' ','_')
    name = name.replace('-','_')
    names.append(name)


transfermarkt_values['id'] = names

for index, row in transfermarkt_values.iterrows():
    transfermarkt_values.at[index, 'id'] = f"{row['id']}_{row['age']}"
    print(transfermarkt_values.at[index, 'id'])

cols = ['id','player','value','club','age'] 
transfermarkt_values = transfermarkt_values[cols]




david_torres_37
manuel_sebastian_36
roberto_navarro_35
alejandro_revuelta_montero_23
moises_parra_gutierrez_21
matthew_craig_20
diego_mendez_20
owen_bevan_19
dexter_lembikisa_19
pablo_munoz_19
george_abbott_18
kami_doyle_18
jack_hinshelwood_18
david_ozoh_18
samuel_ntanda_18
samuel_amo_ameyaw_17
lewis_miley_17
ethan_nwaneri_16
fidel_33
claudio_bravo_40
alvaro_negredo_38
vicente_guaita_36
andrea_consigli_36
angel_rodriguez_36
tim_ream_35
damian_suarez_35
seamus_coleman_34
sergio_asenjo_34
aridane_hernandez_34
luis_hernandez_34
jordi_masip_34
marc_albrighton_33
matt_ritchie_33
joel_robles_33
bram_nuytinck_33
pedro_bigas_33
luigi_sepe_32
borja_garcia_32
alberto_perea_32
jeison_murillo_31
jese_30
kevin_vazquez_30
alex_ferrari_29
nicola_murru_28
curro_27
connor_ronan_25
riccardo_marchizza_25
alberto_del_moral_23
olasagasti_23
lucas_oliveira_rosa_23
diego_collado_22
john_donald_22
kike_salas_21
chem_campbell_20
jayden_braaf_20
franco_carboni_20
pablo_marin_20
marko_milovanovic_20
omari_hutchi

In [34]:
ids = []
for df_key, df in dataframes.items():
    print(df)

                                    Player   Nation    Pos            Squad  \
0        1                Brenden Aaronson   us USA  MF,FW     Leeds United   
1        2                   George Abbott  eng ENG     MF        Tottenham   
2        3                       Che Adams  sct SCO     FW      Southampton   
3        4                     Tyler Adams   us USA     MF     Leeds United   
4        5                Tosin Adarabioyo  eng ENG     DF           Fulham   
5        6                    Nayef Aguerd   ma MAR     DF         West Ham   
6        7                Naouirou Ahamada   fr FRA  MF,FW   Crystal Palace   
7        8                 Rayan Aït Nouri   dz ALG     DF           Wolves   
8        9                 Kristoffer Ajer   no NOR     DF        Brentford   
9       10                   Manuel Akanji   ch SUI     DF  Manchester City   
10      11                      Nathan Aké   nl NED     DF  Manchester City   
11      12                 Marc Albrighton  eng ENG 

In [35]:
ids = []
for df_key, df in dataframes.items():
    for index, row in df.iterrows():
        ids.append( '_'.join(unidecode(df.at[index,'Player']).lower().replace('-', ' ').split()) +'_'+ str(2023 - int(df.at[index,'Born'])))
    print(len(ids))
    print(len(df))
    df['id'] = ids
    ids = []



1768
1768
132
132
1685
1685
1768
1768
1768
1768


In [36]:
for df_key, df in dataframes.items():
    merged_df = pd.merge(df, transfermarkt_values, on='id', how='outer')


    # Filter rows where either 'B' or 'C' is NaN (indicating no match)
    unmatched_rows = merged_df[(merged_df['value'].isna())]


    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)

    unmatched_rows.to_csv(df_key + 'unmatched_rows.csv')

for df_key, df in dataframes.items():
    for index, row in df.iterrows():
        print(df.at[index,'id'])
        if df.at[index,'id'] in unmatched_rows['id'].values:
            df.at[index,'id'] = '_'.join(unidecode(df.at[index,'Player']).lower().replace('-', ' ').split()) +'_'+ str(2023 - int(df.at[index,'Born'])-1)
            print(df.at[index,'id'])

print(dataframes['players_general_stats'].query('Player == "Zlatan Ibrahimović"'))



brenden_aaronson_23
brenden_aaronson_22
george_abbott_18
che_adams_27
tyler_adams_24
tosin_adarabioyo_26
tosin_adarabioyo_25
nayef_aguerd_27
naouirou_ahamada_21
rayan_ait_nouri_22
kristoffer_ajer_25
manuel_akanji_28
nathan_ake_28
marc_albrighton_34
marc_albrighton_33
thiago_alcantara_32
carlos_alcaraz_21
carlos_alcaraz_20
trent_alexander_arnold_25
trent_alexander_arnold_24
alisson_31
alisson_30
dele_alli_27
miguel_almiron_29
julian_alvarez_23
daniel_amartey_29
daniel_amartey_28
samuel_amo_ameyaw_17
joachim_andersen_27
elliot_anderson_21
elliot_anderson_20
jaidon_anthony_24
jaidon_anthony_23
michail_antonio_33
antony_23
cameron_archer_22
cameron_archer_21
alphonse_areola_30
joe_aribo_27
adam_armstrong_26
stuart_armstrong_31
kepa_arrizabalaga_29
kepa_arrizabalaga_28
pierre_emerick_aubameyang_34
ludwig_augustinsson_29
serge_aurier_31
serge_aurier_30
taiwo_awoniyi_26
yasin_ayari_20
yasin_ayari_19
andre_ayew_34
andre_ayew_33
jordan_ayew_32
jordan_ayew_31
luke_ayling_32
cesar_azpilicueta_34


In [37]:
for df_key, df in dataframes.items():
    merged_df = pd.merge(df, transfermarkt_values, on='id', how='inner')
    df.to_csv('C:\\Users\\Jedrek\\Documents\\Inżynierka\\Dane\\'+df_key+'.csv')
    unmatched_rows = merged_df['value']


    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)

    print(len(unmatched_rows))

1715
129
1639
1716
1716


In [38]:
i = 0
df = transfermarkt_values
for i in range(0,len(df['value'])):
    if 'mln' in df['value'][i]:
        print(i)
        df['value'][i] =  df['value'][i].replace('mln','0000')
        df['value'][i] =  df['value'][i].replace(',','')
        df['value'][i] =  df['value'][i].replace(' €','')
        df['value'][i] =  df['value'][i].replace(' ','')
    if 'm' in df['value'][i]:
        df['value'][i] =  df['value'][i].replace('m','0000')
        df['value'][i] =  df['value'][i].replace('.','')
        df['value'][i] =  df['value'][i].replace('€','')
        df['value'][i] =  df['value'][i].replace(' ','')
    if 'tys' in df['value'][i]:
        df['value'][i] =  df['value'][i].replace('tys.','000')
        df['value'][i] =  df['value'][i].replace('€','')
        df['value'][i] =  df['value'][i].replace(' ','')
    if 'k' in df['value'][i]:
        df['value'][i] =  df['value'][i].replace('k','000')
        df['value'][i] =  df['value'][i].replace('€','')
        df['value'][i] =  df['value'][i].replace(' ','')
    if 'tys.' in df['value'][i]:
        df['value'][i] =  df['value'][i].replace('tys.','000')
        df['value'][i] =  df['value'][i].replace('€','')
        df['value'][i] =  df['value'][i].replace(' ','')
    if '-' in df['value'][i]:
        df['value'][i] = ''



transfermarkt_values.to_csv('C:\\Users\\Jedrek\\Documents\\Inżynierka\\Dane\\transfermarkt_values.csv')


1623
1624
1625
1626
1627
1628
1629
1630
1631
1632
1634
1635
1637
1638
1639
1640
1641
1642
1643
1644
1645
1647
1648
1649
1650
1651
1652
1656
1657
1658
1659
1660
1662
1663
1667
1668
1671
1672
